In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install pymorphy2==0.8
!pip install spacy==2.0.12

In [0]:
!git clone https://github.com/buriy/spacy-ru.git #&& cp -r ./spacy-ru/ru2/. /content/gdrive/My Drive/Colab Notebooks/Бермуды/ru2

Cloning into 'spacy-ru'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 231 (delta 7), reused 29 (delta 7), pack-reused 191
Receiving objects: 100% (231/231), 99.21 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stop_words = set(stopwords.words("russian"))
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import spacy
nlp = spacy.load('spacy-ru/ru2')

In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/districts_posts(1).db')

In [0]:
for_proc = []
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM districts")
    rows = cur.fetchall()
 
    for row in rows:
        for_proc.append(row)

In [0]:
con.close()

In [0]:
len(for_proc)

13965

In [0]:
def prepro(text):
    text = re.sub('[.,\?!;:]', ' ; ', text)
    spacy_step = nlp(text)
    temp = [t.lemma_ for t in spacy_step if bool(re.match(r'[а-яА-Я]+$|[а-яА-Я]+\-[а-яА-Я]+$', t.lemma_)) and t.lemma_ not in nltk_stop_words and t.lemma_ != '-PRON-']
    return ' '.join(temp)

In [0]:
from tqdm import tqdm_notebook

In [0]:
updated = []
for post in tqdm_notebook(for_proc):
    post+= tuple([prepro(post[2])])
    updated.append(post)

In [0]:
import sqlite3
from sqlite3 import Error

def sql_connection():
    try:
        con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/districts_clt.db')
        return con
    except Error:
        print(Error)

def sql_table(con):
    cursorObj = con.cursor()
    cursorObj.execute("""CREATE TABLE districts(id integer PRIMARY KEY, district text, post text, cleaned_post text, photo integer, audio integer, video integer, link integer, 
                        poll integer, views integer, comments integer, likes integer, share integer, total_action integer, date text)""")
    con.commit()

con = sql_connection()
sql_table(con)

In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/districts_clt.db')

def sql_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute('''INSERT INTO districts(id, district, post, cleaned_post, photo, audio, video, link, poll, views, comments, 
                        likes, share, total_action, date) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', entities)
    con.commit()

for post in tqdm_notebook(updated):
    if post[-1] != '':
        entities = post[0:3]+tuple([post[-1]])+post[3:-1]
        sql_insert(con, entities)

In [0]:
con.close()

In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/districts_clt.db')

In [0]:
n=0
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM districts")
    rows = cur.fetchall()
 
    for row in rows:
        n+=1
print(n)

9961


13965 до
9961 после